# Ref
* code: https://github.com/Pyomo/pyomo/tree/main/examples/pyomo
* source: https://projects.coin-or.org/Coopr/changeset/9689/

## 1. Greedy Heuristic
* solver1.py

In [4]:
# IOptSolve 불러오는 과정이 자꾸 오류나서 따로 정의

import pyomo

class IOptSolver(Interface):
    """Interface class for creating optimization solvers"""

    def available(self, exception_flag=True):
        """Determine if this optimizer is available."""

    def warm_start_capable(self):
        """ True is the solver can accept a warm-start solution."""

    def solve(self, *args, **kwds):
        """Perform optimization and return an SolverResults object."""

    def reset(self):
        """Reset the state of an optimizer"""

    def set_options(self, istr):
        """Set the options in the optimizer from a string."""

    def __bool__(self):
        """Alias for self.available()"""

In [5]:
from pyomo.core import *
from pyomo.common.plugin import *
from pyomo.opt import *

class MySolver(object):

    alias('greedy')

    # Declare that this is an IOptSolver plugin
    implements(IOptSolver)

    # Solve the specified problem and return
    # a SolverResults object
    def solve(self, instance, **kwds):
        print("Starting greedy heuristic")
        val, instance = self._greedy(instance)
        n = value(instance.N)
        # Setup results
        results = SolverResults()
        results.problem.name = instance.name
        results.problem.sense = ProblemSense.minimize
        results.problem.num_constraints = 1
        results.problem.num_variables = n
        results.problem.num_objectives = 1
        results.solver.status = SolverStatus.ok
        soln = results.solution.add()
        soln.value = val
        soln.status = SolutionStatus.feasible
        for j in sequence(n):
            if instance.y[j].value == 1:
                soln.variable[instance.y[j].name] = {"Value" : 1, "Id" : j}
        return results

    # Perform a greedy search
    def _greedy(self, instance):
        p = value(instance.P)
        n = value(instance.N)
        m = value(instance.M)
        fixed=set()
        # Initialize
        for j in sequence(n):
            instance.y[j].value=0
        # Greedily fix the next best facility
        for i in sequence(p):
            best = None
            ndx=j
            for j in sequence(n):
                if j in fixed:
                    continue
                instance.y[j].value=1
                # Compute value
                val = 0.0
                for kk in sequence(m):
                    tmp=copy.copy(fixed)
                    tmp.add(j)
                    tbest = None
                    for jj in tmp:
                        if tbest == None or instance.d[jj,kk].value < tbest:
                            tbest = instance.d[jj,kk].value
                    val += tbest
                # Keep best greedy choice
                if best == None or val < best:
                    best=val
                    ndx=j
                instance.y[j].value=0
            fixed.add(ndx)
            instance.y[ndx].value=1
        return [best, instance]

## 2. Randomized Heuristic
* solver2.py

In [6]:
# Imports from Pyomo
from pyomo.core import *
from pyomo.common.plugin import *
from pyomo.opt import *
import random
import copy

class MySolver(object):

    alias('random')

    # Declare that this is an IOptSolver plugin
    implements(IOptSolver)

    # Solve the specified problem and return
    # a SolverResults object
    def solve(self, instance, **kwds):
        print("Starting random heuristic")
        val, sol = self._random(instance)
        n = value(instance.N)
        # Setup results
        results = SolverResults()
        results.problem.name = instance.name
        results.problem.sense = ProblemSense.minimize
        results.problem.num_constraints = 1
        results.problem.num_variables = n
        results.problem.num_objectives = 1
        results.solver.status = SolverStatus.ok
        soln = results.solution.add()
        soln.value = val
        soln.status = SolutionStatus.feasible
        for j in sequence(n):
            soln.variable[instance.y[j].name] = {"Value" : sol[j-1], "Id" : j}
        # Return results
        return results

    # Perform a random search
    def _random(self, instance):
        sol = [0]*instance.N.value
        for j in range(instance.P.value):
            sol[j] = 1
        # Generate 100 random solutions, and keep the best
        best = None
        best_sol = []
        for kk in range(100):
            random.shuffle(sol)
            # Compute value
            val=0.0
            for j in sequence(instance.M.value):
                val += min([instance.d[i,j].value
                            for i in sequence(instance.N.value)
                            if sol[i-1] == 1])
            if best == None or val < best:
                best=val
                best_sol=copy.copy(sol)
        return [best, best_sol]

## 3. P-median
* pmedian.py

In [7]:
from pyomo.core import *

def pyomo_create_model(options=None, model_options=None):
    import random

    random.seed(1000)

    model = AbstractModel()

    model.N = Param(within=PositiveIntegers)

    model.Locations = RangeSet(1,model.N)

    model.P = Param(within=RangeSet(1,model.N))

    model.M = Param(within=PositiveIntegers)

    model.Customers = RangeSet(1,model.M)

    model.d = Param(model.Locations, model.Customers, initialize=lambda n, m, model : random.uniform(1.0,2.0), within=Reals)

    model.x = Var(model.Locations, model.Customers, bounds=(0.0,1.0))

    model.y = Var(model.Locations, within=Binary)

    def rule(model):
        return sum( model.d[n,m]*model.x[n,m] for n in model.Locations for m in model.Customers )
    model.obj = Objective(rule=rule)

    def rule(model, m):
        return (sum( model.x[n,m] for n in model.Locations ), 1.0)
    model.single_x = Constraint(model.Customers, rule=rule)

    def rule(model, n, m):
        return (None, model.x[n,m] - model.y[n], 0.0)
    model.bound_y = Constraint(model.Locations, model.Customers, rule=rule)

    def rule(model):
        return (sum( model.y[n] for n in model.Locations ) - model.P, 0.0)
    model.num_facilities = Constraint(rule=rule)

    return model